In [1]:
%pylab inline
import pandas as pd
from keras import layers, models, metrics, optimizers, preprocessing, losses, regularizers
from keras.preprocessing import sequence
import regex
import re
from gensim.models import Word2Vec
from collections import Counter
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
df_all = pd.read_csv(open('training.1600000.processed.noemoticon.csv', encoding='ascii', errors='ignore'), header=None, 
                       names=['pol', 'id', 'date', 'lyx', 'user', 'txt']).append( # Latin-1 is a hack
           pd.read_csv('testdata.manual.2009.06.14.csv', header=None, 
                       names=['pol', 'id', 'date', 'lyx', 'user', 'txt']))
n_train = 1600000
n_test = df_all.shape[0] - n_train

In [3]:
df_all = df_all.loc[df_all.pol != 2]
df_all.pol = (df_all.pol == 4)

In [4]:
n_train = 1600000
n_test = df_all.shape[0] - n_train

In [5]:
date_re = re.compile(r'(?P<week_day>\w{3}) (?P<month>\w{3}) (?P<day>\d\d) (?P<hour>\d\d):(?P<minute>\d\d):(?P<second>\d\d) (?P<tz>\w{3}) (?P<year>\d{4})')

In [6]:
df_all[['week_day', 'day', 'hour']] = \
    df_all.date.str.extract(date_re, expand=True).loc[:,['week_day', 'day', 'hour']]

In [7]:
df_all = pd.get_dummies(df_all, columns=['week_day', 'day', 'hour'])

In [8]:
import regex
df_all.txt = (df_all.txt
              .str.replace(r'https?:\/\/\S+\b|www\.(\w+\.)+\S*', r' <URL> ')
              .str.replace(r'@\w*', r' <USER> ')
              .str.replace(r'#([A-Z0-9\-]+)', r' <HASHTAG> \1 <ALLCAPS> ')
              .str.replace(r'#(\S+)', lambda x: ' <HASHTAG> ' + ' '.join(regex.split(r'(?=[A-Z])', x.group(1))))
              .str.replace(r'<3', r' <HEART> ')
              .str.replace(r'([!?.]){2,}', r' \1 <REPEAT> ')
              .str.replace(r'\b(\S*?)(.)\2{2,}\b', r' \1\2 <ELONG> ')
              .str.replace(r'\s([^a-z0-9()<>\'`\-]){2,}\s',
                           lambda x: x.group(0).lower() + '<ALLCAPS> ')).str.lower()

In [23]:
df_test = df_all[n_train:].copy()
df_train = df_all[:n_train].copy()
del df_all

In [24]:
df_train = df_train.loc[np.random.permutation(df_train.shape[0])]

In [25]:
split_re = re.compile(r'[^\w<>]')

In [26]:
words = Counter(re.split(split_re, df_train.txt.str.cat(sep=' ')))

In [29]:
word_num = {word: i + 1 for word, i in words.most_common(20002)}

In [31]:
def convert_txt(df):
    tweets = []
    for tweet in df.txt:
        current_tweet_list = []
        for word in re.split(split_re, tweet):
            if word not in word_num or words[word] < 5:
                continue
            else:
                current_tweet_list.append(word_num[word])
        tweets.append(np.array(current_tweet_list))
    return tweets

In [32]:
maxlen = 60

In [33]:
X_train = sequence.pad_sequences(convert_txt(df_train), maxlen=maxlen, padding='post')

In [34]:
X_test = sequence.pad_sequences(convert_txt(df_test), maxlen=maxlen, padding='post')

In [38]:
y_test = df_test.pol

In [39]:
y_train = df_train.pol

In [40]:
X_train, X_cv, y_train, y_cv = train_test_split(
    X_train, y_train, test_size=.1)

In [41]:
np.mean(np.count_nonzero(X_train, axis=1))

14.097610416666667

In [42]:
tweet = layers.Input((maxlen,), dtype='int32')

In [157]:
embedded = layers.Embedding(20002, 128, input_length=maxlen,)(tweet)

In [158]:
embedded_normalized = layers.Dropout(.5)(embedded)

In [159]:
conv1 = layers.Convolution1D(256, 5, padding='same', activation='relu')(embedded_normalized)

In [160]:
conv1_dropout = layers.BatchNormalization()(conv1)

In [161]:
conv2 = layers.Convolution1D(256, 4, padding='same', activation='relu')(conv1_dropout)

In [162]:
conv2_dropout = layers.BatchNormalization()(conv2)

In [168]:
lstm = layers.Bidirectional(layers.LSTM(16, recurrent_dropout=.3))(conv2_dropout)

In [169]:
lstm_dropout = layers.BatchNormalization()(lstm)

In [170]:
result = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(1e-3))(lstm_dropout)

In [171]:
model = models.Model(tweet, result)

In [172]:
model.load_weights('conv-conv-bidirlstm.model')

In [31]:
model.compile(optimizer=optimizers.Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train, epochs=20, batch_size=256, 
          validation_data=(X_cv, y_cv))

Train on 1440000 samples, validate on 160000 samples
Epoch 1/20
1440000/1440000 [==============================] - 1215s - loss: 0.3590 - acc: 0.8415 - val_loss: 0.3339 - val_acc: 0.8546
Epoch 2/20
1440000/1440000 [==============================] - 1213s - loss: 0.3586 - acc: 0.8417 - val_loss: 0.3361 - val_acc: 0.8533
Epoch 3/20
1440000/1440000 [==============================] - 1200s - loss: 0.3578 - acc: 0.8421 - val_loss: 0.3389 - val_acc: 0.8522
Epoch 4/20
1440000/1440000 [==============================] - 1196s - loss: 0.3573 - acc: 0.8424 - val_loss: 0.3401 - val_acc: 0.8518
Epoch 5/20
   6144/1440000 [..............................] - ETA: 1165s - loss: 0.3608 - acc: 0.8381

KeyboardInterrupt: 

In [33]:
model.load_weights('lstm-glove.model')

In [ ]:
import keras.

In [34]:
model.history.history

{}

In [173]:
y_pred = model.predict(X_test, batch_size=256, verbose=1)

InvalidArgumentError: indices[1,8] = 68944 is not in [0, 20002)
	 [[Node: embedding_6/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_6/embeddings/read, _arg_input_1_0_1)]]

Caused by op 'embedding_6/Gather', defined at:
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/pshevchuk/twitter-sentiment/venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/pshevchuk/twitter-sentiment/venv/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/pshevchuk/twitter-sentiment/venv/lib/python3.6/sit

In [47]:
X_test[0]

array([  14165,  847053, 1050169,  464094,  330482,  677694,  733354,
        889739,  992092, 1050169,  626053, 1095865,   32850,  889739,
        626053,  828915,  626066,  729819,  484095,  545272,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0], dtype=int32)

In [48]:
from sklearn import metrics

In [49]:
metrics.accuracy_score(y_test, y_pred > .5)

0.84679665738161558

In [50]:
metrics.roc_auc_score(y_test, y_pred)

0.92621220587322273

In [44]:
y_pred = model.predict(X_test, batch_size=256)
print metrics.roc_auc_score(y_test, y_pred)
print metrics.accuracy_score(y_test, y_pred > .5)

0.930371887999
0.844011142061


In [ ]:
print "OK"

In [51]:
model.save_weights('lstm-glove.model.new')

In [64]:
df_train.shape

(1600000, 67)

In [65]:
X_train.shape

(1440000, 60)

In [66]:
X_cv.shape

(160000, 60)

In [35]:
%clear

In [37]:
%clear all

In [70]:
import h5py

In [71]:
f = h5py.File('conv-conv-bidirlstm.model', 'r')

In [78]:
list(f.attrs.keys())

['layer_names', 'backend', 'keras_version']

In [80]:
f.attrs['layer_names']

array([b'input_13', b'embedding_7', b'dropout_11', b'conv1d_13',
       b'batch_normalization_16', b'conv1d_14', b'batch_normalization_17',
       b'bidirectional_3', b'batch_normalization_18', b'dense_25'],
      dtype='|S22')

In [91]:
filtered_layers = [layer for layer in model.layers if getattr(layers, 'weights', None)]

In [92]:
filtered_layers

[]

In [132]:
filtered_layers = []
for layer in model.layers:
    weights = layer.weights
    if weights:
        filtered_layers.append(layer)

In [133]:
filtered_layers

In [127]:
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
filtered_layer_names = []
for name in layer_names:
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if weight_names:
        filtered_layer_names.append(name)

In [128]:
filtered_layer_names

['embedding_7',
 'conv1d_13',
 'batch_normalization_16',
 'conv1d_14',
 'batch_normalization_17',
 'bidirectional_3',
 'batch_normalization_18',
 'dense_25']

In [108]:
for k, name in enumerate(filtered_layer_names):
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    weight_values = [g[weight_name] for weight_name in weight_names]
    print(name, weight_values)

embedding_7 [<HDF5 dataset "embeddings:0": shape (20002, 128), type "<f4">]
conv1d_13 [<HDF5 dataset "kernel:0": shape (5, 128, 256), type "<f4">, <HDF5 dataset "bias:0": shape (256,), type "<f4">]
batch_normalization_16 [<HDF5 dataset "gamma:0": shape (256,), type "<f4">, <HDF5 dataset "beta:0": shape (256,), type "<f4">, <HDF5 dataset "moving_mean:0": shape (256,), type "<f4">, <HDF5 dataset "moving_variance:0": shape (256,), type "<f4">]
conv1d_14 [<HDF5 dataset "kernel:0": shape (4, 256, 256), type "<f4">, <HDF5 dataset "bias:0": shape (256,), type "<f4">]
batch_normalization_17 [<HDF5 dataset "gamma:0": shape (256,), type "<f4">, <HDF5 dataset "beta:0": shape (256,), type "<f4">, <HDF5 dataset "moving_mean:0": shape (256,), type "<f4">, <HDF5 dataset "moving_variance:0": shape (256,), type "<f4">]
bidirectional_3 [<HDF5 dataset "kernel:0": shape (256, 64), type "<f4">, <HDF5 dataset "recurrent_kernel:0": shape (16, 64), type "<f4">, <HDF5 dataset "bias:0": shape (64,), type "<f4">

In [120]:
from keras.engine.topology import preprocess_weights_for_loading

In [124]:
original_keras_version = f.attrs['keras_version'].decode('utf8')
original_backend = f.attrs['backend'].decode('utf8')

In [134]:
for k, name in enumerate(filtered_layer_names):
        g = f[name]
        weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
        weight_values = [g[weight_name] for weight_name in weight_names]
        print(k)
        layer = filtered_layers[k]
        symbolic_weights = layer.weights
        try:
            weight_values = preprocess_weights_for_loading(layer,
                                                           weight_values,
                                                           original_keras_version,
                                                           original_backend)
        except Exception as e:
            print(name, 'failed')
        


0
1
conv1d_13 failed
2
3
4
5
6
7


In [ ]:
fil